# TSA Chapter 2: Python Lab: Simulate and Fit AR(1)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch2/TSA_ch2_python_simulate/TSA_ch2_python_simulate.ipynb)

This notebook demonstrates:
- Simulate AR(1) with phi = 0
- 6 for 300 observations
- Plot series and ACF/PACF, fit model, compare true vs estimated phi, Ljung-Box residual diagnostics

In [ ]:
!pip install matplotlib numpy scipy statsmodels pandas -q

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from matplotlib.patches import Polygon
# Style configuration
COLORS = {
    'blue': '#1A3A6E',
    'red': '#DC3545',
    'green': '#2E7D32',
    'orange': '#E67E22',
    'gray': '#666666',
    'purple': '#8E44AD',
}

plt.rcParams.update({
    'axes.facecolor': 'none',
    'figure.facecolor': 'none',
    'savefig.transparent': True,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.grid': False,
    'font.size': 9,
    'axes.titlesize': 10,
    'axes.labelsize': 9,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'legend.fontsize': 8,
    'figure.dpi': 150,
    'lines.linewidth': 1.2,
    'axes.edgecolor': '#333333',
    'axes.linewidth': 0.8,
})

np.random.seed(42)

def save_chart(fig, name):
    """Save chart as PDF and PNG."""
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}.pdf + .png')

In [ ]:
np.random.seed(42)

# Parameters
n = 300
phi_true = 0.6
sigma = 1.0

print("=" * 60)
print("PYTHON EXERCISE 1: SIMULATE AND FIT AR(1)")
print("=" * 60)

# Step 1: Simulate AR(1)
eps = np.random.normal(0, sigma, n)
x = np.zeros(n)
for t in range(1, n):
    x[t] = phi_true * x[t-1] + eps[t]

print(f"\nSimulated AR(1) with φ = {phi_true}, n = {n}")

# Step 2: Plot series and ACF/PACF

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
axes[0].plot(x, color='steelblue', linewidth=0.8)
axes[0].set_title('Simulated AR(1) Series')
axes[0].set_xlabel('t')

plot_acf(x, ax=axes[1], lags=20, alpha=0.05)
axes[1].set_title('ACF')

plot_pacf(x, ax=axes[2], lags=20, alpha=0.05)
axes[2].set_title('PACF')

plt.tight_layout()
save_chart(fig, 'sem2_ar1_simulation')
plt.show()

# Step 3: Fit AR(1)
model = ARIMA(x, order=(1, 0, 0)).fit()
print(model.summary())

phi_hat = model.params[1]
print(f"\nTrue φ = {phi_true}")
print(f"Estimated φ̂ = {phi_hat:.4f}")
print(f"Estimation error = {abs(phi_hat - phi_true):.4f}")

# Step 4: Residual diagnostics
resid = model.resid
lb_test = acorr_ljungbox(resid, lags=[10], return_df=True)
print(f"\nLjung-Box test (lag 10):")
print(f"  Q-statistic = {lb_test['lb_stat'].values[0]:.4f}")
print(f"  p-value = {lb_test['lb_pvalue'].values[0]:.4f}")

if lb_test['lb_pvalue'].values[0] > 0.05:
    print("  → Fail to reject H₀: residuals are white noise ✓")
else:
    print("  → Reject H₀: residuals have autocorrelation ✗")